In [1]:
import os
import time
import io
import hashlib
import numpy as np
from PIL import Image
from utils import parseBBOXFile, getTagValue

In [2]:
#os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

In [3]:
import tensorflow as tf
from object_detection.utils import dataset_util, label_map_util

In [4]:
label_map = label_map_util.load_labelmap('../annotations/label_map.pbtxt')
label_map_dict = label_map_util.get_label_map_dict(label_map)

Convert a folder of images + xml files to TF-record format, needed for training.

In [39]:
def convertFolder(name, image_path, annotate_path, image_format = b'jpg'):
    files = set([filename[:-4] for filename in os.listdir(image_path)])
    if len(files) == 0:
        return
    
    print("Creating TF-record:", '{}/{}.record'.format(annotate_path, name))
    with tf.io.TFRecordWriter('{}/{}.record'.format(annotate_path, name)) as writer:
        for f in files:
            img, xml = '{}/{}.jpg'.format(image_path, f), '{}/{}.xml'.format(image_path, f)
            encoded_jpg = None
            with tf.io.gfile.GFile(img, 'rb') as fid:
                encoded_jpg = fid.read()

            image = Image.open(io.BytesIO(encoded_jpg))
            key = hashlib.sha256(encoded_jpg).hexdigest().encode('utf8')
            width, height = image.size
            filename = f.encode('utf8')
            xmins, xmaxs = [], []
            ymins, ymaxs = [], []
            classes_name, classes_id = [], []
            truncated = []
            poses = []
            difficult = []

            for obj in parseBBOXFile(xml):
                lbl = getTagValue(obj, 'name')

                xmin = (float(getTagValue(obj, 'xmin')) / width)
                xmax = (float(getTagValue(obj, 'xmax')) / width)

                ymin = (float(getTagValue(obj, 'ymin')) / height)
                ymax = (float(getTagValue(obj, 'ymax')) / height)

                classes_name.append(lbl.encode('utf8'))
                classes_id.append(label_map_dict[lbl])
                xmins.append(xmin)
                xmaxs.append(xmax)
                ymins.append(ymin)
                ymaxs.append(ymax)
                
                truncated.append(int(getTagValue(obj, 'truncated')))
                difficult.append(int(getTagValue(obj, 'difficult')))
                poses.append(getTagValue(obj, 'pose').encode('utf8'))

            record = tf.train.Example(features=tf.train.Features(feature={
                'image/height': dataset_util.int64_feature(height),
                'image/width': dataset_util.int64_feature(width),
                'image/filename': dataset_util.bytes_feature(filename),
                'image/source_id': dataset_util.bytes_feature(filename),
                'image/key/sha256': dataset_util.bytes_feature(key),
                'image/encoded': dataset_util.bytes_feature(encoded_jpg),
                'image/format': dataset_util.bytes_feature(image_format),
                'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
                'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
                'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
                'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
                'image/object/class/text': dataset_util.bytes_list_feature(classes_name),
                'image/object/class/label': dataset_util.int64_list_feature(classes_id),
                'image/object/difficult': dataset_util.int64_list_feature(difficult),
                'image/object/truncated': dataset_util.int64_list_feature(truncated),
                'image/object/view': dataset_util.bytes_list_feature(poses),
            })).SerializeToString()
            writer.write(record)
            print("Wrote", f, 'to the record.')
            #break
    print("Finished!")

In [40]:
convertFolder('train', '../images/train', '../annotations')

Creating TF-record: ../annotations/train.record
Wrote e39_aug_s_07-01-13-27 to the record.
Wrote e39_aug_s_07-01-16-05 to the record.
Wrote e39_aug_n_07-01-15-23 to the record.
Wrote e39_aug_n_08-01-13-05 to the record.
Wrote e39_aug_n_08-01-11-42 to the record.
Wrote e39_aug_n_08-01-15-06 to the record.
Wrote e39_aug_n_08-01-22-09 to the record.
Wrote e39_aug_n_06-01-23-00 to the record.
Wrote e39_aug_n_07-01-11-25 to the record.
Wrote e39_aug_n_07-01-22-40 to the record.
Wrote e39_aug_n_06-01-19-24 to the record.
Wrote e39_aug_s_07-01-22-25 to the record.
Wrote e39_aug_n_07-01-00-14 to the record.
Wrote e39_aug_n_06-01-19-19 to the record.
Wrote e39_aug_n_08-01-22-59 to the record.
Wrote e39_aug_n_07-01-18-26 to the record.
Wrote e39_aug_n_08-01-19-27 to the record.
Wrote e39_aug_n_08-01-12-21 to the record.
Wrote e39_aug_n_08-01-19-41 to the record.
Wrote e39_aug_n_07-01-16-54 to the record.
Wrote e39_aug_n_07-01-12-39 to the record.
Wrote e39_aug_s_19-01-13-01 to the record.
Wrote 

Wrote e39_vatland_10-01-17-19 to the record.
Wrote e39_aug_s_07-01-20-42 to the record.
Wrote rv150_ullevaal_17-01-22-20 to the record.
Wrote e39_aug_s_07-01-19-07 to the record.
Wrote e39_aug_n_08-01-18-02 to the record.
Wrote e39_aug_s_08-01-15-53 to the record.
Wrote e39_aug_n_07-01-17-20 to the record.
Wrote e39_aug_s_07-01-12-25 to the record.
Wrote e39_aug_n_06-01-18-49 to the record.
Wrote e39_aug_n_08-01-14-23 to the record.
Wrote e39_aug_s_08-01-17-55 to the record.
Wrote e39_aug_s_07-01-22-31 to the record.
Wrote e6_nostvet_11-01-20-57 to the record.
Wrote e6_nostvet_12-01-12-32 to the record.
Wrote e39_aug_n_07-01-20-07 to the record.
Wrote e39_aug_n_07-01-14-23 to the record.
Wrote e6_osen_09-01-12-33 to the record.
Wrote e39_aug_n_08-01-21-41 to the record.
Wrote e39_aug_n_08-01-11-10 to the record.
Wrote e6_nostvet_18-01-00-18 to the record.
Wrote rv150_ullevaal_17-01-14-40 to the record.
Wrote e6_nostvet_11-01-18-01 to the record.
Wrote e39_aug_n_06-01-19-13 to the recor

Wrote e6_osen_10-01-15-43 to the record.
Wrote e39_aug_n_08-01-21-22 to the record.
Wrote e39_aug_n_08-01-22-48 to the record.
Wrote e39_aug_s_08-01-13-47 to the record.
Wrote e39_aug_n_08-01-14-54 to the record.
Wrote e39_aug_s_08-01-22-05 to the record.
Wrote e39_aug_n_07-01-23-25 to the record.
Wrote e39_aug_s_07-01-12-20 to the record.
Wrote e39_aug_s_06-01-22-06 to the record.
Wrote e39_aug_n_07-01-19-40 to the record.
Wrote e39_aug_s_08-01-12-08 to the record.
Wrote e39_aug_n_07-01-00-07 to the record.
Wrote e39_aug_s_08-01-17-28 to the record.
Wrote e39_aug_n_07-01-12-47 to the record.
Wrote rv150_ullevaal_19-01-14-42 to the record.
Wrote e39_aug_n_06-01-20-49 to the record.
Wrote e39_aug_n_08-01-15-50 to the record.
Wrote e39_aug_s_08-01-15-59 to the record.
Wrote e39_aug_n_08-01-21-06 to the record.
Wrote e39_aug_s_08-01-16-58 to the record.
Wrote e39_aug_s_06-01-22-13 to the record.
Wrote e39_aug_s_07-01-18-53 to the record.
Wrote e39_aug_s_07-01-21-58 to the record.
Wrote e3

Wrote e39_aug_n_08-01-19-09 to the record.
Wrote e39_aug_s_07-01-17-12 to the record.
Wrote e39_aug_s_08-01-18-34 to the record.
Wrote e39_aug_n_19-01-00-05 to the record.
Wrote e39_aug_s_06-01-17-53 to the record.
Wrote e39_aug_n_07-01-13-26 to the record.
Wrote e6_nostvet_11-01-19-14 to the record.
Wrote e39_aug_n_07-01-13-40 to the record.
Wrote rv150_ullevaal_17-01-16-20 to the record.
Wrote e39_aug_s_08-01-12-02 to the record.
Wrote e39_aug_n_07-01-12-00 to the record.
Wrote e39_aug_n_08-01-15-42 to the record.
Wrote e39_aug_n_07-01-13-04 to the record.
Wrote e6_nostvet_18-01-18-02 to the record.
Wrote e39_aug_n_08-01-19-24 to the record.
Wrote e39_aug_s_18-01-16-23 to the record.
Wrote rv150_ullevaal_17-01-18-40 to the record.
Wrote e39_aug_n_08-01-18-08 to the record.
Wrote e39_aug_s_08-01-14-32 to the record.
Wrote e39_aug_n_08-01-19-39 to the record.
Wrote e39_aug_s_08-01-12-27 to the record.
Wrote e39_aug_n_08-01-18-49 to the record.
Wrote e39_aug_s_08-01-12-24 to the record.

In [41]:
convertFolder('test', '../images/test', '../annotations')

Creating TF-record: ../annotations/test.record
Wrote rv150_ullevaal_17-01-15-59 to the record.
Wrote e39_aug_n_07-01-19-50 to the record.
Wrote e6_nostvet_11-01-17-46 to the record.
Wrote e39_aug_s_08-01-19-40 to the record.
Wrote e39_aug_n_07-01-20-26 to the record.
Wrote e39_aug_n_07-01-22-32 to the record.
Wrote e39_vatland_08-01-21-16 to the record.
Wrote e39_aug_s_07-01-21-07 to the record.
Wrote rv150_ullevaal_18-01-14-01 to the record.
Wrote e39_aug_n_08-01-21-52 to the record.
Wrote e39_aug_n_08-01-11-20 to the record.
Wrote e39_aug_n_07-01-23-00 to the record.
Wrote e39_aug_s_06-01-23-32 to the record.
Wrote rv150_ullevaal_19-01-14-01 to the record.
Wrote e39_aug_s_08-01-19-08 to the record.
Wrote e39_aug_s_08-01-22-53 to the record.
Wrote e39_aug_s_06-01-20-41 to the record.
Wrote e39_aug_n_19-01-13-20 to the record.
Wrote e39_aug_n_07-01-18-31 to the record.
Wrote e39_aug_s_08-01-15-19 to the record.
Wrote e39_aug_n_08-01-14-50 to the record.
Wrote e39_aug_s_07-01-15-22 to t